In [41]:
import psycopg2
import pandas as pd
import keys

In [42]:
letters = ['A','A']
price = ['0', '2']
period = ['1-21-2021', '2-21-2021']
trend_period = ['12-21-2020']

In [43]:
alphabets_in_lowercase=[]
for i in range(65,91):
    alphabets_in_lowercase.append(chr(i))
abc = alphabets_in_lowercase
print(abc)
ind_1 = abc.index(letters[0])
ind_2 = abc.index(letters[1]) + 1
abc = abc[ind_1:ind_2]
abc = str(abc).replace('[', '').replace(']', '')
print(abc)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
'A'


In [44]:
#abc = get_abc_arr(lets)
#query = f"select * from historical_stock_data where ticker_id in (select id from master_ticker_list where left(ticker, 1) in ('A', 'b'));"
query = f"select * from historical_stock_data as a where ticker_id in (select id from master_ticker_list where left(ticker, 1) in ({abc})) and '{period[0]}'::date < date::date and date::date < '{period[1] if trend_period != '' else trend_period}'::date limit 1000;"

In [45]:
conn = psycopg2.connect(database="postgres", user=keys.user, password=keys.password, host=keys.host, port="5432")
cur = conn.cursor()
#cur.execute(f"select * from master_ticker_list limit 5;")
cur.execute(query)
data = cur.fetchall()
#print(data)
# ticker_id |        open        |       close        |        high        |        low        |  volume  |        date 
df=pd.DataFrame(data,columns=['ticker_id','open', 'close', 'high','low','volume', 'date'])
print(df)
conn.commit()
conn.close()

      ticker_id                open               close                high  \
0    1040820796  3.2799999713897705  3.2899999618530273   3.299999952316284   
1    1040820796  3.2799999713897705  3.2899999618530273   3.299999952316284   
2    1040820796  3.2899999618530273  3.2799999713897705   3.299999952316284   
3    1040820796  3.2699999809265137  3.2699999809265137  3.2899999618530273   
4    1040820796  3.2799999713897705  3.2799999713897705  3.2899999618530273   
..          ...                 ...                 ...                 ...   
995   648094622  20.729999542236328  20.899999618530273  21.079999923706055   
996   648094622  20.940000534057617  21.049999237060547  21.049999237060547   
997   648094622   20.06999969482422  20.329999923706055  20.950000762939453   
998   648094622  20.420000076293945  20.479999542236328  20.610000610351562   
999   648094622  20.389999389648438    20.6299991607666  20.709999084472656   

                    low     volume                 

In [46]:
print(len(df))
ids = set(df['ticker_id'].to_list())
for _id in ids:
    ser = df[df['ticker_id'] == _id]
    maxed = max(ser['close'])
    mined = min(ser['close'])
    #print(mined,maxed)
    if mined > int(price[0]) or maxed < int(price[1]):
        print(mined, maxed)
        df.drop(ser.index, inplace=True)
print(len(df))
#print(df)
#rem = df[ df['ticker_id']isin(ind) ]
#print(rem)
# Delete these row indexes from dataFrame
#df.drop(indexNames , inplace=True)
#print(df)

1000
29.979999542236328 32.47999954223633
9.020000457763672 16.31999969482422
22.950000762939453 26.549999237060547
3.509999990463257 19.899999618530273
2.450000047683716 3.0299999713897705
6.550000190734863 6.550000190734863
10.890000343322754 11.5
6.570000171661377 8.039999961853027
2.369999885559082 3.2100000381469727
55.06999969482422 58.849998474121094
153.3300018310547 161.75
19.170000076293945 21.049999237060547
38.13999938964844 45.310001373291016
55.52000045776367 67.33999633789062
36.56999969482422 44.630001068115234
77.25 87.55000305175781
44.810001373291016 59.83000183105469
286.6600036621094 314.1600036621094
9.489999771118164 10.789999961853027
0.6600000262260437 0.75
4.159999847412109 8.5
93.26000213623047 128.1199951171875
13.770000457763672 16.649999618530273
3.25 3.299999952316284
53.40999984741211 69.41000366210938
23.149999618530273 29.350000381469727
12.869999885559082 15.789999961853027
10.119999885559082 14.920000076293945
88.5 94.9800033569336
10.100000381469727